In [1]:
import numpy as np

In [2]:
class TicTacToeEnvironment():
    def __init__(self, mode='X', points_for_tie=False):
        self.mode = mode
        self.table = np.zeros((3,3), dtype=np.uint8)
        self.observation_space = self.table.reshape(-1)
        self.points_for_tie = False
    def checkwin(self):
        for i in range(3):
            if self.table[0,i] == self.table[1,i] and self.table[1,i] == self.table[2,i] and self.table[1,i] != 0:
                return True, self.table[0,i]
            if self.table[i, 0] == self.table[i,1] and self.table[i,1] == self.table[i,2] and self.table[i,1] != 0:
                return True, self.table[i, 0]
        if self.table[0,0] == self.table[1,1] and self.table[1,1] == self.table[2,2] and self.table[0,0] != 0:
            return True, self.table[1,1]
        if self.table[0,2] == self.table[1,1] and self.table[1,1] == self.table[2,0] and self.table[0,2] != 0:
            return True, self.table[1,1]
        return False, 0
    def board_full(self):
        return not 0 in self.table
    def encode_table(self):
        flat_table = self.table.reshape(-1)
        encoding = ''
        for i in range(len(flat_table)):
            encoding += str(int(flat_table[i]))
        return encoding
    def observation(self):
        return self.encode_table()
    def reset(self):
        self.table = np.zeros((3,3))
        return self.encode_table()
    def encoding_to_idx(self, encoding):
        values = np.zeros(9)
        for i in range(9):
            values[i] = encoding[i]
        return int(np.sum(values* 3**np.arange(9)))
    def step(self, action, player):
        if action < 0 or action  >= 9:
            raise ValueError('action space an int in range [0,9)')
        valid_move = False
        to_place = 1 if player == 'X' else 2
        if self.table[action // 3, action % 3] == 0:
            self.table[action // 3, action % 3] = to_place
            valid_move = True
        is_win, winner_num = self.checkwin()
        board_full = self.bloard_full()
        terminated = True if (is_win or board_full) else False
        if is_win and winner_num == to_place:
            #print('player', player, 'won!')
            reward = 1
        else if winner_num == 2 and board_full and self.points_for_tie:
            reward = 0.5
        else:
            reward = 0
        return self.encode_table(), reward, terminated, board_full
    def print_board(self):
        char_table = np.array([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
        for i in range(3):
            for j in range(3):
                if self.table[i,j] == 1:
                    char_table[i,j] = 'X'
                elif self.table[i,j] == 2:
                    char_table[i,j] = 'O'
            
        print(char_table[0,0], '|', char_table[0,1], '|', char_table[0,2])
        print('---------')
        print(char_table[1,0], '|', char_table[1,1], '|', char_table[1,2])
        print('---------')
        print(char_table[2,0], '|', char_table[2,1], '|', char_table[2,2])

    def play_against_policy(self, policy, mode='X', deterministic=True):
        player_piece = 1 if mode == 'X' else 2
        ai_piece = 2 if mode == 'X' else 1
        for i in range(9):
            self.print_board()
            if (mode == 'X' and (i % 2) == 0) or (mode == 'O' and (i % 2) == 1):
                print('choose your move')
                move = -1
                while (move < 0 or move >= 9):
                    move = int(input())
                self.table[move // 3, move % 3] = player_piece
            else:
                print('press enter for ai move')
                input()
                if deterministic:
                    ai_move = np.argmax(policy[self.encoding_to_idx(self.encode_table())])
                ##sample from Q-values as if they were probability distribution
                else:
                    dist = policy[self.encoding_to_idx(self.encode_table())]
                    #turn to probability distribution
                    dist_probs = dist / np.sum(dist)
                    ##sample the action
                    ai_move = np.random.choice(np.arange(9), p=dist_probs)
                self.table[ai_move // 3, ai_move % 3] = ai_piece
            game_over, _ = self.checkwin()
            if game_over:
                break
        self.print_board()            

In [3]:
np.random.choice([0,1], p=[0.5,0.5])

1

In [4]:
env = TicTacToeEnvironment()
env.print_board()

  |   |  
---------
  |   |  
---------
  |   |  


In [5]:
def encoding_to_idx(encoding):
    values = np.zeros(9)
    for i in range(9):
        values[i] = encoding[i]
    return int(np.sum(values* 3**np.arange(9)))

In [6]:
def epsilon_greedy(epsilon, player_table, string_state):
    valid_indices = []
    for i in range(9):
        if string_state[i] == '0':
            valid_indices.append(i)
    if len(valid_indices) < 1:
        raise ValueError('No valid board choices!?')
    random_action = np.random.choice([True,False], p=[epsilon, (1-epsilon)])
    if random_action:
        return np.random.choice(valid_indices)
    
    valid_choices = player_table[encoding_to_idx(string_state)][valid_indices]
    max_valid = valid_indices[np.argmax(valid_choices)]
    return max_valid

In [7]:
def train(X_player, O_player, env, num_episodes, lr, epsilon_decay, gamma, start_decay):
    x_wins_arr = []
    o_wins_arr = []
    ties_arr = []
    x_wins = 0
    o_wins = 0
    ties = 0
    players = [X_player, O_player]
    epsilon = 1
    for episode in range(num_episodes):
        x_up_next_state = env.reset()
        terminated = False
        turn_num = 0
        while (True):
            
            ##X action            
            x_action = epsilon_greedy(epsilon, X_player, x_up_next_state)
            x_just_played_state, x_reward, x_terminated, board_full = env.step(x_action, 'X')
            
            
            ##bellman O update -> can't update on turn 0 yet, O hasn't played
            if turn_num > 0:
                O_player[env.encoding_to_idx(x_play_last_turn), o_action] += \
                lr * ((o_reward + gamma*np.max(O_player[env.encoding_to_idx(x_just_played_state)]) \
                       - O_player[env.encoding_to_idx(x_play_last_turn), o_action]))
                
                if o_terminated or board_full:
                    o_wins+= 1 if o_reward == 1 else 0
                    if board_full and not o_reward:
                        ties+=1
                    break
            x_play_last_turn = x_just_played_state
            
            ##O action
            o_action = epsilon_greedy(epsilon, O_player, x_just_played_state)
            o_just_played_state, o_reward, o_terminated, board_full = env.step(o_action, 'O')
            

            ##bellman X update
            X_player[env.encoding_to_idx(x_up_next_state), x_action] += \
            lr * ((x_reward + gamma*np.max(X_player[env.encoding_to_idx(o_just_played_state)]) \
                   - X_player[env.encoding_to_idx(x_up_next_state), x_action]))
            
            if x_terminated or board_full:
                x_wins+= 1 if x_reward == 1 else 0
                break
        
            x_up_next_state = o_just_played_state
            turn_num += 1
        if episode > start_decay:
            epsilon = np.exp(-epsilon_decay * (episode - start_decay))
        if ((episode + 1) % 10000) == 0:
            print('Episode', episode + 1, 'done')
            print(f'epsilon: {epsilon:.3f}')
            print('X Wins:', x_wins)
            print('O Wins:', o_wins)
            print('Ties:', ties)
            print()
            x_wins_arr.append(x_wins)
            o_wins_arr.append(o_wins)
            ties_arr.append(ties)
    return x_wins_arr, o_wins_arr, ties_arr

In [8]:
##policies
X_player = np.zeros((3**9, 9))
O_player = np.zeros((3**9, 9))

##environment
env = TicTacToeEnvironment()

##hyperparams
num_episodes = 1000000
lr = 0.2
gamma = 1
start_decay = 300000
epsilon_decay = 5 / (num_episodes - start_decay)

In [ ]:
x_wins_arr, o_wins_arr, ties_arr = train(X_player, O_player, env, num_episodes, lr, epsilon_decay, gamma, start_decay)

Episode 10000 done
epsilon: 1.000
X Wins: 3500
O Wins: 2896
Ties: 3604

Episode 20000 done
epsilon: 1.000
X Wins: 7119
O Wins: 5817
Ties: 7064

Episode 30000 done
epsilon: 1.000
X Wins: 10664
O Wins: 8665
Ties: 10671

Episode 40000 done
epsilon: 1.000
X Wins: 14366
O Wins: 11526
Ties: 14108

Episode 50000 done
epsilon: 1.000
X Wins: 17897
O Wins: 14391
Ties: 17712

Episode 60000 done
epsilon: 1.000
X Wins: 21445
O Wins: 17237
Ties: 21318

Episode 70000 done
epsilon: 1.000
X Wins: 25012
O Wins: 20161
Ties: 24827

Episode 80000 done
epsilon: 1.000
X Wins: 28558
O Wins: 23068
Ties: 28374

Episode 90000 done
epsilon: 1.000
X Wins: 32132
O Wins: 26028
Ties: 31840

Episode 100000 done
epsilon: 1.000
X Wins: 35719
O Wins: 28923
Ties: 35358

Episode 110000 done
epsilon: 1.000
X Wins: 39259
O Wins: 31791
Ties: 38950

Episode 120000 done
epsilon: 1.000
X Wins: 42792
O Wins: 34692
Ties: 42516

Episode 130000 done
epsilon: 1.000
X Wins: 46413
O Wins: 37584
Ties: 46003

Episode 140000 done
epsilon:

In [ ]:
test_env = TicTacToeEnvironment()
state = test_env.reset()

In [ ]:
# ##policies
# X_player = np.zeros((3**9, 9))
# O_player = np.zeros((3**9, 9))

In [ ]:
test_env.reset()
test_env.play_against_policy(O_player, mode='X', deterministic=True)

In [ ]:
test_env.reset()
test_env.play_against_policy(X_player, mode='O', deterministic=True)

In [ ]:
np.max(O_player)

In [ ]:
np.min(O_player)

In [ ]:
np.argmax(X_player[encoding_to_idx('0100111100')])

In [ ]:
encoding_to_idx('100000000')

In [ ]:
O_player[encoding_to_idx('01000001111')]

In [ ]:
np.argmax(O_player)

In [ ]:
X_player[encoding_to_idx('121212000')]

In [ ]:
X_player[encoding_to_idx('121212100')]

In [ ]:
X_player[encoding_to_idx('121212000')]